# Introduction to `fastplotlib`

This notebook goes through basic components of the `fastplotlib` API including images, image updates, and gridplots.

In [1]:
import fastplotlib as fpl
from ipywidgets import VBox, HBox, IntSlider
import numpy as np
from sidecar import Sidecar

### Simple image

In [2]:
# create a `Plot` instance
plot = fpl.Plot()

# make some random 2D image data
data = np.random.rand(512, 512)

# plot the image data
image_graphic = plot.add_image(data=data, name="random-image")

# display plot in side car
sc = Sidecar(title="simple image")

with sc:
    # show the plot
    display(plot.show())

RFBOutputContext()

/home/clewis7/repos/fastplotlib/fastplotlib/graphics/_features/_base.py:34: UserWarning: converting float64 array to float32
  warn(f"converting {array.dtype} array to float32")


### Use the handle on the bottom right corner of the _canvas_ to resize it. You can also pan and zoom using your mouse!

Changing graphic "features"

In [3]:
image_graphic.cmap = "viridis"

In [4]:
image_graphic.data = 0

In [5]:
image_graphic.data[::15, :] = 1
image_graphic.data[:, ::15] = 1

In [6]:
image_graphic.data = np.random.rand(512, 512)

### Plots are indexable and give you their graphics by name

In [7]:
plot.graphics

(<weakproxy at 0x7f9e4f700220 to ImageGraphic at 0x7f9e49989e90>,)

In [8]:
plot["random-image"]

<weakproxy at 0x7f9e4f700220 to ImageGraphic at 0x7f9e49989e90>

#### The `Graphic` instance is also returned when you call `plot.add_<graphic_type>`.

In [9]:
image_graphic

<weakproxy at 0x7f9e4f700220 to ImageGraphic at 0x7f9e49989e90>

In [10]:
image_graphic == plot["random-image"]

True

In [11]:
# close sidecar
sc.close()

### Image updates

This examples show how you can define animation functions that run on every render cycle.

In [12]:
# create another `Plot` instance
plot_v = fpl.Plot()

plot.canvas.max_buffered_frames = 1

# make some random data again
data = np.random.rand(512, 512)

# plot the data
plot_v.add_image(data=data, name="random-image")

# a function to update the image_graphic
# a plot will pass its plot instance to the animation function as an arugment
def update_data(plot_instance):
    new_data = np.random.rand(512, 512)
    plot_instance["random-image"].data = new_data

#add this as an animation function
plot_v.add_animations(update_data)

# sidecar to display plot
sc = Sidecar(title="simple image")

with sc:
    # show the plot
    display(plot_v.show())

RFBOutputContext()

### We can share controllers across plots

This example creates a new plot, but it synchronizes the pan-zoom controller

In [13]:
plot_sync = fpl.Plot(controller=plot_v.controller)

data = np.random.rand(512, 512)

image_graphic_instance = plot_sync.add_image(data=data, cmap="viridis")

# you will need to define a new animation function for this graphic
def update_data_2():
    new_data = np.random.rand(512, 512)
    # alternatively, you can use the stored reference to the graphic as well instead of indexing the Plot
    image_graphic_instance.data = new_data

plot_sync.add_animations(update_data_2)

plot_sync.show()

RFBOutputContext()

#### Keeping a reference to the Graphic instance, as shown above `image_graphic_instance`, is useful if you're creating something where you need flexibility in the naming of the graphics

In [14]:
sc.close()

### You can also use `ipywidgets.VBox` and `HBox` to stack plots.

In [15]:
VBox([plot_v.show(), plot_sync.show()])

In [22]:
HBox([plot_v.show(), plot_sync.show()])

In [16]:
# close plots
plot_v.close()
plot_sync.close()
plot.close()

# GridPlot

Allows for proper subplotting. Create graphics and add them to a `GridPlot`.

In [18]:
# GridPlot of shape 2 x 3 with all controllers synced
grid_plot = fpl.GridPlot(shape=(2, 3), controllers="sync")

# Make a random image graphic for each subplot
for subplot in grid_plot:
    # create image data
    data = np.random.rand(512, 512)
    # add an image to the subplot
    subplot.add_image(data, name="rand-img")

# Define a function to update the image graphics with new data
# add_animations will pass the gridplot to the animation function
def update_data(gp):
    for sp in gp:
        new_data = np.random.rand(512, 512)
        # index the image graphic by name and set the data
        sp["rand-img"].data = new_data
        
# add the animation function
grid_plot.add_animations(update_data)

sc = Sidecar(title="gridplot")

# show the gridplot 
with sc:
    display(grid_plot.show())

RFBOutputContext()

## Accessing subplots within `GridPlot`

In [19]:
# positional indexing
# row 0 and col 0
grid_plot[0, 0]

unnamed: Subplot @ 0x7f9df45ad150
  parent: None
  Graphics:
	'rand-img': ImageGraphic @ 0x7f9dec665990

### You can get the graphics within a subplot, just like with simple `Plot`

In [20]:
grid_plot[0, 1].graphics

(<weakproxy at 0x7f9e2c10b060 to ImageGraphic at 0x7f9dec50ead0>,)

### and change their properties

In [22]:
grid_plot[0, 1].graphics[0].cmap.vmax = 0.5

### more indexing with `GridPlot`

In [23]:
# you can give subplots human-readable string names
grid_plot[0, 2].name = "top-right-plot"

In [24]:
grid_plot["top-right-plot"]

top-right-plot: Subplot @ 0x7f9df45df510
  parent: None
  Graphics:
	'rand-img': ImageGraphic @ 0x7f9dec514890

In [25]:
# view its position
grid_plot["top-right-plot"].position

(0, 2)

In [26]:
# these are really the same
grid_plot["top-right-plot"] is grid_plot[0, 2]

True

Indexing with subplot name and graphic name

In [27]:
grid_plot["top-right-plot"]["rand-img"].cmap.vmin = 0.5

In [28]:
sc.close()